# Filtrado y Análisis Automatizado de CVs usando IA
### Proyecto de Inteligencia Artificial y Principios Técnicos

Este notebook demuestra el pipeline de análisis automatizado de currículums utilizando procesamiento de lenguaje natural, matching semántico y clasificación supervisada.

---

**Autores:**  
- Morales Chunga Carlos  
- Cardenas Iglesias Hugo  
- Mauricci Becerra Leandro  
- Meza Cordova Aryel  

---

## **Conclusiones**

- El sistema desarrollado permite automatizar la revisión y preselección de currículums de forma objetiva, transparente y reproducible, facilitando los procesos de reclutamiento en áreas técnicas.
- El uso combinado de búsqueda por palabras clave, análisis semántico y clasificación de perfiles técnicos mejora la precisión frente a métodos tradicionales, identificando tanto coincidencias explícitas como afinidad contextual.
- La arquitectura modular facilita la adaptación a diferentes vacantes, ampliación de criterios de filtrado, y entrenamiento en distintos dominios técnicos.
- Este proyecto es una base sólida para integrar análisis más avanzados, como filtros de soft skills, pruebas técnicas automáticas y evaluación psicométrica.
- La herramienta puede ser adaptada por equipos de recursos humanos para reducir el sesgo humano y acelerar la selección de talento alineado a las necesidades específicas del puesto.

---



### Instrucciones generales

1. Instala dependencias: `pip install -r requirements.txt`
2. Genera los datos de prueba (CVs y etiquetas)
3. Entrena el modelo de IA para clasificación de perfiles técnicos
4. Ejecuta el pipeline de análisis y filtrado desde este notebook
---


In [ ]:
# Solo ejecuta esta celda si NO tienes las dependencias instaladas.
# (Puedes hacerlo por terminal: pip install -r requirements.txt)

# !pip install pdfplumber scikit-learn sentence-transformers pandas fpdf


In [35]:
# Generar datos de ejemplo (CVs PDF y etiquetas)
import os

ruta_cvs = 'data/cvs_pdfs'
ruta_etiquetas = 'data/etiquetas.csv'

if not (os.path.exists(ruta_cvs) and os.path.exists(ruta_etiquetas)):
    print("Generando CVs y etiquetas de ejemplo...")
    os.system("python tools/generador_cvs_etiquetados.py")
else:
    print("Los archivos de CVs y etiquetas ya existen.")


Generando CVs y etiquetas de ejemplo...


In [37]:
# Entrenamiento del modelo de clasificación de perfiles técnicos
if not (os.path.exists('classifier.pkl') and os.path.exists('vectorizer.pkl')):
    print("Entrenando clasificador de perfiles técnicos...")
    os.system("python tools/entrenar_clasificador.py")
else:
    print("El clasificador ya está entrenado.")


El clasificador ya está entrenado.


In [38]:
# Importación de módulos propios y utilidades
import sys
sys.path.append('./src')

from utils import extract_and_clean_all_pdfs, normalize_keyword, keywords_score
from embedding import compute_semantic_scores
from classifier import load_classifier, predict_profiles
import pandas as pd


In [39]:
pdf_folder = 'data/cvs_pdfs'
cv_texts = extract_and_clean_all_pdfs(pdf_folder)
print(f"CVs cargados y limpiados: {len(cv_texts)}")


CVs cargados y limpiados: 200


In [40]:
keywords_input = input(
    "Ingresa las palabras clave separadas por coma (ejemplo: python, sql, docker, ingles(nativo), .net):\n"
)
keywords = [normalize_keyword(k) for k in keywords_input.split(',')]
print("Palabras clave normalizadas:", keywords)


Palabras clave normalizadas: ['linux', 'docker', 'redes', 'cloud computing', 'aws', 'azure', 'servidores', 'aleman(intermedio)', 'ingles(nativo)']


In [41]:
kw_results = keywords_score(cv_texts, keywords)

# Ejemplo de muestra rápida:
sorted_kw = sorted(kw_results.items(), key=lambda x: x[1][0], reverse=True)
for pdf, (score, matches) in sorted_kw[:5]:
    print(f"{pdf}: {score} puntos")
    print("Coincidencias:", matches)


cv_157.pdf: 8 puntos
Coincidencias: ['linux', 'docker', 'redes', 'cloud computing', 'aws', 'azure', 'servidores', 'aleman(intermedio)']
cv_162.pdf: 7 puntos
Coincidencias: ['linux', 'docker', 'redes', 'cloud computing', 'aws', 'azure', 'servidores']
cv_163.pdf: 6 puntos
Coincidencias: ['redes', 'cloud computing', 'aws', 'azure', 'servidores', 'ingles(nativo)']
cv_167.pdf: 6 puntos
Coincidencias: ['docker', 'redes', 'cloud computing', 'aws', 'azure', 'ingles(nativo)']
cv_168.pdf: 6 puntos
Coincidencias: ['linux', 'docker', 'redes', 'cloud computing', 'aws', 'servidores']


In [42]:
job_description = input("Pega aquí la descripción de la vacante o perfil deseado:\n")

semantic_scores = compute_semantic_scores(cv_texts, job_description)
semantic_dict = dict(semantic_scores)


In [43]:
clf, vectorizer = load_classifier()
profiles = predict_profiles(cv_texts, clf, vectorizer)


In [44]:
tabla = []

for pdf in cv_texts.keys():
    kw_score, found_keywords = kw_results.get(pdf, (0, []))
    sem_score = semantic_dict.get(pdf, 0)
    perfil = profiles.get(pdf, "Desconocido")
    puntaje_total = kw_score + sem_score
    tabla.append({
        "CV": pdf,
        "Score Keywords": kw_score,
        "Score Semántico": round(sem_score, 3),
        "Perfil Técnico Predicho": perfil,
        "Puntaje Total": round(puntaje_total, 3),
        "Keywords Encontradas": found_keywords
    })

df = pd.DataFrame(tabla)
df = df.sort_values("Puntaje Total", ascending=False).reset_index(drop=True)

from IPython.display import display
display(df[["CV", "Score Keywords", "Score Semántico", "Perfil Técnico Predicho", "Puntaje Total"]].head(10))

# Mostrar palabras clave encontradas por CV (Top 10)
print("\nPalabras clave encontradas por CV (Top 10):")
for idx, row in df.head(10).iterrows():
    print(f"{row['CV']}: {', '.join(row['Keywords Encontradas']) if row['Keywords Encontradas'] else '(ninguna)'}")


,CV,Score Keywords,Score Semántico,Perfil Técnico Predicho,Puntaje Total
0,cv_157.pdf,8,0.514,infraestructura,8.514
1,cv_162.pdf,7,0.540,infraestructura,7.540
2,cv_168.pdf,6,0.539,infraestructura,6.539
3,cv_179.pdf,6,0.535,infraestructura,6.535
4,cv_174.pdf,6,0.518,infraestructura,6.518
5,cv_167.pdf,6,0.514,infraestructura,6.514
6,cv_184.pdf,6,0.514,infraestructura,6.514
7,cv_192.pdf,6,0.513,infraestructura,6.513
8,cv_178.pdf,6,0.511,infraestructura,6.511
9,cv_198.pdf,6,0.506,infraestructura,6.506



Palabras clave encontradas por CV (Top 10):
cv_157.pdf: linux, docker, redes, cloud computing, aws, azure, servidores, aleman(intermedio)
cv_162.pdf: linux, docker, redes, cloud computing, aws, azure, servidores
cv_168.pdf: linux, docker, redes, cloud computing, aws, servidores
cv_179.pdf: linux, docker, redes, cloud computing, azure, servidores
cv_174.pdf: docker, redes, cloud computing, aws, servidores, aleman(intermedio)
cv_167.pdf: docker, redes, cloud computing, aws, azure, ingles(nativo)
cv_184.pdf: linux, docker, redes, cloud computing, azure, servidores
cv_192.pdf: docker, redes, cloud computing, aws, servidores, aleman(intermedio)
cv_178.pdf: linux, redes, cloud computing, aws, azure, servidores
cv_198.pdf: linux, docker, redes, aws, azure, servidores


In [45]:
try:
    cupos = int(input("\n¿Cuántos cupos necesitas cubrir?: "))
    print(f"\nLos {cupos} candidatos mejor rankeados son:")
    display(df.head(cupos))
except Exception as e:
    print("No se seleccionó un número de cupos, mostrando top 10 por defecto.")
    display(df.head(10))



Los 20 candidatos mejor rankeados son:


,CV,Score Keywords,Score Semántico,Perfil Técnico Predicho,Puntaje Total,Keywords Encontradas
0,cv_157.pdf,8,0.514,infraestructura,8.514,"[linux, docker, redes, cloud computing, aws, a..."
1,cv_162.pdf,7,0.540,infraestructura,7.540,"[linux, docker, redes, cloud computing, aws, a..."
2,cv_168.pdf,6,0.539,infraestructura,6.539,"[linux, docker, redes, cloud computing, aws, s..."
3,cv_179.pdf,6,0.535,infraestructura,6.535,"[linux, docker, redes, cloud computing, azure,..."
4,cv_174.pdf,6,0.518,infraestructura,6.518,"[docker, redes, cloud computing, aws, servidor..."
5,cv_167.pdf,6,0.514,infraestructura,6.514,"[docker, redes, cloud computing, aws, azure, i..."
6,cv_184.pdf,6,0.514,infraestructura,6.514,"[linux, docker, redes, cloud computing, azure,..."
7,cv_192.pdf,6,0.513,infraestructura,6.513,"[docker, redes, cloud computing, aws, servidor..."
8,cv_178.pdf,6,0.511,infraestructura,6.511,"[linux, redes, cloud computing, aws, azure, se..."
9,cv_198.pdf,6,0.506,infraestructura,6.506,"[linux, docker, redes, aws, azure, servidores]"



---

### **Interpretación de resultados**

- **Score Keywords:** Número de palabras clave encontradas en el CV.
- **Score Semántico:** Grado de similitud semántica (0 a 1) entre el CV y la vacante.
- **Perfil Técnico Predicho:** Etiqueta asignada por el modelo de IA entrenado.
- **Puntaje Total:** Suma de Score Keywords y Score Semántico. (Puedes ajustar el peso si lo deseas).
- **Palabras clave encontradas:** Revisa el print debajo de la tabla.

Puedes personalizar el pipeline cambiando las palabras clave o la descripción de vacante, para adaptarlo a distintas búsquedas.


#### **Ejemplo de palabras clave y descripciones de vacante**

- **Backend:**  
  Palabras clave: python, java, sql, c#, .net, docker, linux, ingles(nativo)  
  Descripción: Empresa fintech busca Ingeniero/a de Sistemas para desarrollo Backend. Requisitos: dominio avanzado de python, java, sql y .net, experiencia en administración de servidores linux y uso de contenedores docker. Se requiere ingles nativo.

- **Frontend:**  
  Palabras clave: html, css, javascript, react, angular, figma, ingles(nativo), italiano(nativo)  
  Descripción: Startup de e-commerce busca Desarrollador/a Frontend con experiencia en html, css y javascript, frameworks modernos como react o angular, y diseño con figma. Ingles nativo e italiano nativo serán altamente valorados.

- **Data:**  
  Palabras clave: python, sql, machine learning, pandas, power bi, data science, excel avanzado, frances(basico)  
  Descripción: Corporación internacional busca Analista de Datos para área de inteligencia de negocio. Imprescindible: dominio de python, machine learning, pandas y sql. Visualización con power bi y manejo de excel avanzado. Nivel básico de francés valorado.

- **Infraestructura:**  
  Palabras clave: linux, docker, redes, cloud computing, aws, azure, servidores, aleman(intermedio), ingles(nativo)  
  Descripción: Empresa de tecnología busca Especialista en Infraestructura y Cloud. Requisitos: experiencia con linux, docker, redes, servidores, cloud computing (aws, azure). Se requiere ingles nativo y se valorará alemán intermedio.
